In [13]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
# imports
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [5]:
calculator = "https://www.selflessclothes.com/blog/sustainability-calculator/?"

In [6]:
goal = "https://www.selflessclothes.com/blog/sustainability-calculator/?material=COTTON&material=POLYESTER&percentage=50&percentage=50"

In [7]:
item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

In [8]:
material_params = ["COTTON", "RECYCLED_COTTON", "ORGANIC_COTTON", "POLYESTER", "RECYCLED_POLYESTER", "NYLON", "RECYCLED_NYLON", "ACRYLIC", "SPANDEX", "FLAX", "LINEN", "HEMP", "CUPRO", "LYOCELL", "TENCEL_LYOCELL_LENZING", "REFIBRA_TENCEL_LYOCELL_LENZING", "MODAL", "TENCEL_MODAL_LENZING", "VISCOSE", "VISCOSE_BAMBOO", "VISCOSE_ASIA_LENZING", "VISCOSE_EU_LENZING", "SILK", "ALPACA", "WOOL", "RECYCLED_WOOL", "CASHMERE", "RECYCLED_CASHMERE"]

In [9]:
materials_and_percents = {key: value for key, value in item.items() if key != 'item'}
materials_and_percents

{'silk': 53, 'linen': 33, 'cotton': 14}

In [10]:
goal_params = []
for material, percent in materials_and_percents.items():
    if material.upper() in material_params:
        goal_params.extend([f"material={material.upper()}", f"percentage={percent}"])

# Reorder the goal_params list to have materials first
reordered_params = [goal_params[i] for i in range(0, len(goal_params), 2)] + [goal_params[i] for i in range(1, len(goal_params), 2)]

goal_url = calculator + "&".join(reordered_params)
print(goal_url)

https://www.selflessclothes.com/blog/sustainability-calculator/?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14


In [11]:
def get_score_using_selenium(goal_url):
    # Configure Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode

    # Initialize Chrome WebDriver with configured options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Load the page
        driver.get(goal_url)

        # Wait for the score element to be present
        score_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ml-2"))
        )

        # Extract the score text
        score = score_element.text.strip()

        return score
    except Exception as e:
        print("Error:", e)
        return None
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
# goal_url = "https://www.selflessclothes.com/blog/sustainability-calculator/?material=COTTON&material=POLYESTER&percentage=50&percentage=50"
score = get_score_using_selenium(goal_url)
if score:
    print("Score:", score)
else:
    print("Score not found")

Score: (0.4)


In [12]:
def get_material_score(item):
    # List of accepted parameters by the calculator
    material_params = ["COTTON", "RECYCLED_COTTON", "ORGANIC_COTTON", 
                    "POLYESTER", "RECYCLED_POLYESTER", "NYLON", 
                    "RECYCLED_NYLON", "ACRYLIC", "SPANDEX", 
                    "FLAX", "LINEN", "HEMP", "CUPRO", 
                    "LYOCELL", "TENCEL_LYOCELL_LENZING", 
                    "REFIBRA_TENCEL_LYOCELL_LENZING", "MODAL", 
                    "TENCEL_MODAL_LENZING", "VISCOSE", "VISCOSE_BAMBOO", 
                    "VISCOSE_ASIA_LENZING", "VISCOSE_EU_LENZING",
                    "SILK", "ALPACA", "WOOL", "RECYCLED_WOOL", 
                    "CASHMERE", "RECYCLED_CASHMERE"]
    
    # Base calculator URL
    calculator = "https://www.selflessclothes.com/blog/sustainability-calculator/?"

    # Extract the materials out of the item dictionary for calculator URL's query parameters
    materials_and_percents = {key: value for key, value in item.items() if key != 'item'}

    goal_params = []

    # Check if the materials are in the material parameters that the website uses
    for material, percent in materials_and_percents.items():
        if material.upper() in material_params:
            goal_params.extend([f"material={material.upper()}", f"percentage={percent}"])

    # Reorder the parameters so they match the score calculator
    reordered_params = [goal_params[i] for i in range(0, len(goal_params), 2)] + [goal_params[i] for i in range(1, len(goal_params), 2)]

    # String concatenation of the score calculator url
    score_calculator = calculator + "&".join(reordered_params)

    # Configure Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode

    # Initialize Chrome WebDriver with configured options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Load the score calculator page with the results
        driver.get(score_calculator)

        # Wait for the score element to be present
        score_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ml-2"))
        )

        # Extract the score text
        score_text = score_element.text.strip()

        # Extract the score from the parentheses and convert it to a float
        material_score = float(score_text[score_text.find('(') + 1: score_text.find(')')])

        # Returns the clothing item's name and material score
        return (item.get("item"), f"Material Score: {material_score}")
    except Exception as e:
        print("Error:", e)
        return None
    finally:
        # Close the WebDriver
        driver.quit()

# Using BS4

In [16]:
# create session
session = requests.Session()
# retry five times in case of exception
retry = Retry(connect=5, backoff_factor=0.5)
# apply delays between attempts
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)

In [22]:
def bs4_material_calculator(url):
    try:
        material_calculator = session.get(url)

        # content
        content = BeautifulSoup(material_calculator.text, 'html.parser')
        content_lower = str(content).lower()
        print(content_lower)
        # extract score
        # print(content.find_all("div", class_="ml-2 text-muted"))
    except Exception as e:
        return e

In [23]:
url = "https://www.selflessclothes.com/blog/sustainability-calculator/?material=COTTON&material=POLYESTER&percentage=50&percentage=50"
bs4_material_calculator(url)

<!doctype html>

<html data-n-head="%7b%22lang%22:%7b%22ssr%22:%22en%22%7d%7d" data-n-head-ssr="" lang="en">
<head>
<title>sustainability calculator | selfless clothes</title><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width,initial-scale=1" data-n-head="ssr" name="viewport"/><meta content="want to know the impact of your favorite garments? let's find out! the footprint calculator is a tool for sustainability geeks that uses industry knowledge and lifecycle data from manufacturing to help you assess textiles and raise awareness." data-hid="description" data-n-head="ssr" name="description"/><meta content="sustainability calculator" data-hid="og:title" data-n-head="ssr" property="og:title"/><meta content="want to know the impact of your favorite garments? let's find out! the footprint calculator is a tool for sustainability geeks that uses industry knowledge and lifecycle data from manufacturing to help you assess textiles and raise awareness." data-hid="og:descr